# Python for AI Projects

## Introduction

**Generative AI and Agentic AI**

In this Jupyter notebook - we'll quickly setup our Python environment and see how we can run a series of Streamlit dashboards within our Google Colab workspace.

**⚠️ BEFORE YOU BEGIN!**

Make sure you’re using the **GPU runtime** in Google Colab for better performance when running local language models like TinyLlama.

To enable GPU runtime, please go to the menu:  
**Runtime → Change runtime type → Hardware accelerator → GPU**

### Executing Code Cells

To execute each cell in this notebook - you can click on the play button on the left of each cell or hit `command/shift + enter` when you're navigating to the cell.

### Setup and Installation

In the following cell we'll run a few commands to install the required Python packages and also grab all of our Streamlit application code and data artefacts from our GitHub repository.

The cell below should around ~2 minutes to run as we'll need to download and cache a few large-language models from Hugging Face!

In [ ]:
# ====================
# Initial setup steps
# ====================

# Install Python libraries
!pip install --quiet faiss-cpu==1.11.0
!pip install --quiet ctransformers==0.2.27
!pip install --quiet dotenv==0.9.9
!pip install --quiet pyngrok==7.2.12
!pip install --quiet streamlit==1.47.1

# Clone GitHub repo into a "data" folder
!git clone https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259.git data

# Need to change directory into "data" to download git lfs data objects
%cd data
!git lfs pull

# Then we need to change directory back up so all our paths are correct
%cd ..

# Turn off future warnings for cleaner output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Frontload the necessary LLM and AI libraries we'll be using in this tutorial
from sentence_transformers import SentenceTransformer
from huggingface_hub import hf_hub_download, snapshot_download
import os

cache_root = "data/models"

# this cache_dir is important as we'll reuse this in some of our Streamlit apps
tiny_llama_model_path = hf_hub_download(
    repo_id="TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
    filename="tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
    cache_dir=cache_root
)

# Download and cache our embedding model
embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
cached_embedding_model_path = snapshot_download(
  repo_id=embedding_model_id,
  cache_dir=cache_root
)

# We'll use this path to refer to our cached embedding model in our Streamlit apps
print(cached_embedding_model_path)
print("✅ Initial setup steps complete!")

# 1. Introduction to Streamlit

Welcome to this hands-on tutorial! In this notebook, we’ll be using Streamlit, a lightweight Python framework for building interactive web dashboards and AI apps — all with minimal code.

## 1.1 What is Streamlit?

Streamlit lets you turn any Python script into a shareable web app using simple commands like st.write(), st.text_input(), and st.button().

It's perfect for:

- Visualizing data and model predictions
- Prototyping dashboards
- Building AI-powered tools quickly

## 1.2 The Challenge in Google Colab

Because Colab runs on remote virtual machines, we can’t directly access localhost ports like we would on our own computers.

That means even if we run:

```python
streamlit run app.py
```

We still won't be able to visit localhost:8501 in our browser to view our application!

## 1.3 Ngrok for Secure Tunnels

To solve this, we’ll use Ngrok, a tool that creates a secure tunnel from the internet to your Colab machine.

We’ll use the following pattern for all of our Streamlit apps in this tutorial:

- Start running our Streamlit apps on port `8051` (we’ll avoid `8501` to reduce potential conflicts)
- Use Ngrok to expose port 8051 to a public web URL
- Visit the generated public URL to view our live dashboards

# 2. Pre-flight Checks

We will need to setup a few secure access components before we can proceed with our Streamlit and AI agents tutorial.

## 2.1 Setup Ngrok Account

1. Create Ngrok account at https://dashboard.ngrok.com/signup
2. Acquire your `AuthToken` at https://dashboard.ngrok.com/get-started/your-authtoken

![ngrok-copy-auth-token](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/ngrok-copy-auth-token.png)

## 2.2 Setup OpenRouter Account

1. Create OpenRouter account at https://openrouter.ai/sign-up
2. Create an API token at https://openrouter.ai/settings/keys with **read only** access and spend limit set to $0

![open-router-create-api-key](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/open-router-create-api-key.png)

3. Store this API key safely as you won't be able to see it again once you click away

> **WARNING 🚨🚨🚨** don't be like me and share your key publicly like in this image! This example is just for illustration and I've already deleted this set of API keys on my account!!!!!!! 🥵

![open-router-copy-api-key](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/open-router-copy-api-key.png)

## 2.3 Streamlit Password

We'll also secure our Streamlit apps by using a simple authentication password. This ensures that our publicly available URL still has a layer of authentication in-memory so only we can access it.

Simply decide on a simple password for your Streamlit applications - but note that you'll need to type this when we start viewing our dashboards, so I wouldn't make it too long or complicated for a better learning experience!

## 2.4 Setup Authentication

Run the code cell immediately below to populate a local `.env` file with your sensitive information we setup in the previous step!

Make sure you have the following ready so you can paste it in when prompted in the following cell:

* Ngrok Auth Token
* OpenRouter API Key
* Streamlit password (remember to keep it short as we'll need to type it for every Streamlit app we run!)

We’ll ask for these values securely using getpass, and store them in a local `.env` file so they’re not exposed in the notebook.

> **Warning 🚨🚨🚨** Make sure to never share your Auth and API keys and be careful to avoid exposing them to the public especially when commiting files into GitHub! I've also setup a secure cleaup step to make sure we remove the `.env` file at [bottom of this notebook!](#final-steps)

In [ ]:
# =========================
# Secure .env Setup Script
# =========================
import getpass
import os
import dotenv

# Step 1: Check if .env file exists
if os.path.exists(".env"):
    print("✅ .env file already exists!")

    # Load existing environment variables
    dotenv.load_dotenv()

    # Check each required variable individually
    ngrok_token = os.getenv("NGROK_AUTH_TOKEN")
    if not ngrok_token:
        ngrok_token = getpass.getpass("🔐 Paste your NGROK_AUTH_TOKEN: ")
        with open(".env", "a") as f:
            f.write(f'NGROK_AUTH_TOKEN={ngrok_token}\n')

    openrouter_key = os.getenv("OPEN_ROUTER_API_KEY")
    if not openrouter_key:
        openrouter_key = getpass.getpass("🔐 Paste your OPEN_ROUTER_API_KEY: ")
        with open(".env", "a") as f:
            f.write(f'OPEN_ROUTER_API_KEY={openrouter_key}\n')

    streamlit_password = os.getenv("STREAMLIT_PASSWORD")
    if not streamlit_password:
        password_input = getpass.getpass("🔐 Set your STREAMLIT_PASSWORD (press Enter to use default): ")
        streamlit_password = password_input or "linkedin-learning"
        if password_input == "":
            print(f"ℹ️ No password entered — using default: {streamlit_password}")
        else:
            print("✅ Streamlit password set!")
        with open(".env", "a") as f:
            f.write(f'STREAMLIT_PASSWORD={streamlit_password}\n')

    print("✅ All required environment variables are now set!")

# Step 2: If .env does not exist, prompt for everything
else:
    print("⚙️ .env file not found — let's create one now!")

    ngrok_token = getpass.getpass("🔐 Paste your NGROK_AUTH_TOKEN: ")
    openrouter_key = getpass.getpass("🔐 Paste your OPEN_ROUTER_API_KEY: ")
    password_input = getpass.getpass("🔐 Set your STREAMLIT_PASSWORD (press Enter to use default): ")

    streamlit_password = password_input or "linkedin-learning"
    if password_input == "":
        print(f"ℹ️ No password entered — using default: {streamlit_password}")
    else:
        print("✅ Streamlit password set! (hidden)")

    with open(".env", "w") as f:
        f.write(f'NGROK_AUTH_TOKEN={ngrok_token}\n')
        f.write(f'OPEN_ROUTER_API_KEY={openrouter_key}\n')
        f.write(f'STREAMLIT_PASSWORD={streamlit_password}\n')

    print("✅ .env file created securely!")

# Step 3: Append model paths to .env
with open(".env", "a") as f:
    f.write(f'TINY_LLAMA_MODEL_PATH={tiny_llama_model_path}\n')
    f.write(f'EMBEDDING_MODEL_PATH={cached_embedding_model_path}\n')
print("✅ Local model paths appended to .env file!")

# Finally - load in the newly created environment variables
dotenv.load_dotenv()
print("✅ Latest .env file successfully loaded!")

# 3. Streamlit Hello World App

In the next section, we’ll demonstrate how to run a basic Streamlit app to help verify everything is working smoothly before we dive deeper into our GenAI Streamlit applications.

## 3.1 Streamlit Helper Functions

We'll firstly define some Python helper functions to help us launch and shutdown our Streamlit applications using Ngrok to host our live dashboards.

We'll also include one function to allow us to inspect the source code for our Streamlit application code.

You can run the cell directly below to initialize these functions:

* `launch_streamlit_in_colab`
* `shutdown_streamlit_and_ngrok`
* `render_python_file_as_markdown`

In [ ]:
def launch_streamlit_in_colab(app_path: str):
    """
    Launch a Streamlit app within Google Colab using an ngrok tunnel.

    Parameters:
    ----------
    app_path : str
        Path to the Streamlit `.py` app file to be launched (e.g., "app.py")

    Returns:
    -------
    public_url : str
        A publicly accessible ngrok URL that points to the Streamlit app running on port 8501.

    This function performs:
    ------------------------
    - Authenticates with ngrok using a token stored in the .env file (`NGROK_AUTH_TOKEN`)
    - Kills any existing ngrok tunnels
    - Launches a new tunnel to port 8501
    - Starts the Streamlit app in the background
    - Waits briefly to allow the app to start up
    """

    import os
    import time
    from pyngrok import ngrok

    # ----------------------------
    # Set ngrok authentication token
    # ----------------------------
    ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))

    # ----------------------------
    # Kill any existing tunnels
    # ----------------------------
    ngrok.kill()

    # ----------------------------
    # Open a new tunnel to port 8501
    # ----------------------------
    public_url = ngrok.connect(8501, "http")
    print(f"✅ Streamlit app will be live at: {public_url}")

    # ----------------------------
    # Start the Streamlit app in the background
    # ----------------------------
    os.system(f"streamlit run {app_path} &>/content/logs.txt &")

    # ----------------------------
    # Wait a few seconds to ensure the app starts
    # ----------------------------
    time.sleep(3)

    return public_url

def shutdown_streamlit_and_ngrok():
    """
    Shut down any running Streamlit app and terminate all ngrok tunnels in a Colab session.

    This function performs:
    ------------------------
    - Kills all active ngrok tunnels using the pyngrok API
    - Terminates any background Streamlit processes using `pkill`
    - Provides feedback via console print statements
    """

    import os
    from pyngrok import ngrok

    print("⛔ Shutting down Streamlit and ngrok...")

    # ----------------------------
    # Kill all active ngrok tunnels
    # ----------------------------
    try:
        ngrok.kill()
        print("✅ ngrok tunnel(s) killed.")
    except Exception as e:
        print(f"⚠️ Error killing ngrok: {e}")

    # ----------------------------
    # Kill all background Streamlit processes
    # ----------------------------
    os.system("pkill -f streamlit")
    print("✅ Streamlit processes killed.")
    
def render_python_file_as_markdown(path: str):
    """
    Render the contents of a Python script as syntax-highlighted Markdown in a Colab or Jupyter notebook.

    Parameters:
    ----------
    path : str
        File path to the Python script to be rendered (e.g., "app.py")

    Returns:
    -------
    IPython.display.Markdown
        A Markdown object that displays the script with Python syntax highlighting.
    """

    from IPython.display import Markdown

    # ----------------------------
    # Read file contents
    # ----------------------------
    with open(path, "r") as f:
        code = f.read()

    # ----------------------------
    # Format and return as Markdown code block
    # ----------------------------
    return Markdown(f"```python\n{code}\n```")


## 3.2 Running our Streamlit Application

Once we run the `launch_streamlit_in_colab("data/streamlit-hello-world.py")` command in the following cell and you click on the shared URL link - we should see the following screens (if all things go well!)

> ✅ Make sure to click on the URL which looks like this: `"https://<some-random-string>.ngrok-free.app"` instead of `"http://localhost:8501"`

After clicking on this URL - you should see this welcome page from Ngrok which might seem alarming but it's fine to click on `Visit Site` to proceed!

![streamlit-hello-world-ngrok](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-ngrok.png)

You should first be welcomed by an authentication page.

![streamlit-hello-world-login](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-login.png)

Then following this - you should see text input asking for a prompt.

![streamlit-hello-world-prompt](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-prompt.png)

Finally - if all goes well, you should see something like this output - try it out using your own prompts!

![streamlit-hello-world-success](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-success.png)

After you're done with this hello world example - you can run the cell directly below to stop running the Streamlit application and continue with our tutorial!

In [ ]:
# Launch the Streamlit app in Colab
launch_streamlit_in_colab("data/streamlit-hello-world.py")

## 3.3 Shutdown Streamlit and Ngrok

Once you're ready to move onto the next component - we can shutdown our Streamlit app and free up the Ngrok tunnel.

In [ ]:
# Shutdown the Streamlit app in Colab
shutdown_streamlit_and_ngrok()

## 3.4 View Streamlit App Source Code

The source code for the streamlit app is below with lots of code comments to clearly highlight every step of the way!

In [ ]:
render_python_file_as_markdown("data/streamlit-hello-world.py")

# 4. Pure Python LLM App

In the next cell - we'll run a Streamlit app which details a simple `Retrieval-Augmented-Generation` (RAG) pipeline using pure Python only.

This will help us set the stage before we start applying higher-level abstractions using the `LangChain` and `LangGraph` frameworks to modularize our application.

**Placeholder for Exclalidraw diagram**

In [ ]:
# Placeholder for pure Python app

# 5. LangChain App

In the next cell - we'll run a Streamlit app which extends our pure Python app by replacing certain components using `LangChain`

**Placeholder for Exclalidraw diagram**

In [ ]:
# Placeholder for LangChain app

# 5. LangGraph Agentic AI

In the next cell - we'll run a Streamlit app which further extends upon our static AI workflows using `LangChain` to an autonomous agentic AI framework using `LangGraph`

**Placeholder for Exclalidraw diagram**

In [ ]:
# Placeholder for LangGraph app

In [ ]:
# Cleanup - kill any ngrok tunnels and delete sensitive .env file
import os
import dotenv
from pyngrok import ngrok

if os.path.exists(".env"):
    
    # Load environment variables from .env file
    dotenv.load_dotenv()

    # Check if NGROK_AUTH_TOKEN is set and stop any existing tunnels
    if os.getenv("NGROK_AUTH_TOKEN"):
        ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))

        # Stop any existing Ngrok tunnels
        ngrok.kill()
        print("✅ Ngrok tunnels stopped.")
    
    # Remove the .env file
    os.remove(".env")
    print("✅ .env file removed safely.")